# Introduction to LangChain and RAG

Welcome to the first part of our workshop! In this session, we'll explore how to build AI-powered applications using **LangChain**, a popular framework for developing applications with Large Language Models (LLMs). We'll start with a simple chatbot and then enhance it with Retrieval Augmented Generation (RAG).

## Setting Up Our Environment

First, we need to set up our environment. We'll use OpenAI's models, so we need an API key. You can define your `OPENAI_API_KEY` in the `.env` file.

The code retrieve the key and sets some global configurations:
- `LLM_MODEL`: The specific model we'll use
- `LLM_TEMPERATURE`: Controls randomness in responses (0 means very deterministic)

In [191]:
import os

In [192]:
if not os.environ.get("OPENAI_API_KEY"):
    raise ValueError("Please set OPENAI_API_KEY environment variable")

LLM_MODEL = "gpt-4o-mini"
LLM_TEMPERATURE = 0

## Building a Simple ChatBot

Let's start with creating a basic chatbot using **LangChain**. We'll use:
- `ChatOpenAI`: The interface to OpenAI's chat models
- `SystemMessage`: Defines the bot's behavior and role
- `HumanMessage`: Represents user input

Our chatbot will act as a Financial Analyst. We'll create it by:
1. Instantiating the model
2. Defining a system prompt that sets the bot's role
3. Sending a user query and getting a response with `.invoke()`

This demonstrates the basic pattern of LLM interactions: prompt → response.

In [193]:
from IPython.display import Markdown
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_openai import ChatOpenAI

In [194]:
# Create a ChatOpenAI instance with the LLM model and temperature
base_model = ChatOpenAI(model=LLM_MODEL, temperature=LLM_TEMPERATURE)

In [195]:
BASE_PROMPT = """
You are a Financial Analyst. Do your best to help the client with their request based on your expertise. Give a succinct and clear response.
"""

In [196]:
# Request from the client
request = "I want to invest in the technology sector. Can you please define an investment strategy?"

# Message list for the base model
messages = [
    SystemMessage(BASE_PROMPT),
    HumanMessage(request),
]

# Invoke the model with the messages
response = base_model.invoke(messages)

In [197]:
Markdown(response.content)

Certainly! Here’s a succinct investment strategy for the technology sector:

1. **Research and Identify Sub-Sectors**: Focus on specific areas within technology, such as cloud computing, artificial intelligence, cybersecurity, or renewable energy tech. Each sub-sector has different growth potentials and risks.

2. **Diversification**: Invest in a mix of large-cap, mid-cap, and small-cap companies to balance risk and reward. Consider ETFs or mutual funds that focus on technology for broader exposure.

3. **Fundamental Analysis**: Evaluate companies based on key metrics such as revenue growth, profit margins, P/E ratios, and market share. Look for companies with strong balance sheets and innovative products.

4. **Stay Informed**: Keep up with industry trends, technological advancements, and regulatory changes. Subscribe to tech news outlets and follow market analysts.

5. **Risk Management**: Set a clear risk tolerance and use stop-loss orders to protect your investments. Consider the volatility of tech stocks and adjust your portfolio accordingly.

6. **Long-Term Perspective**: Technology can be volatile in the short term. Focus on long-term growth potential and be prepared to hold investments through market fluctuations.

7. **Regular Review and Rebalance**: Periodically assess your portfolio’s performance and make adjustments based on market conditions and your investment goals.

By following this strategy, you can position yourself to capitalize on growth opportunities in the technology sector while managing risk effectively.

## Understanding Retrieval Augmented Generation (RAG)

Now comes the exciting part! RAG is a technique that enhances LLM responses by giving them access to external knowledge. Instead of relying solely on the model's training data, we can provide relevant information from our own database.

### Vector Database Setup

To implement RAG, we need:
1. A collection of documents (in our case, a currated set of 1'000 articles from Bloomberg financial news)
2. A way to search these documents efficiently (vector database)
3. A function to retrieve relevant information based on queries

Here we use:
- `Chroma`: A vector database for storing and retrieving documents
- `OpenAIEmbeddings`: Converts text into vector representations

Let's first set up the global configuration for our retriever.

In [198]:
EMBEDDING_MODEL = "text-embedding-3-small"
RETRIEVAL_K = 50

We'll then define helper functions to load our documents and store them in a vector store.

In [199]:
import pickle

from langchain_chroma import Chroma
from langchain_core.documents import Document
from langchain_openai import OpenAIEmbeddings

In [200]:
def load_documents(pickle_filepath: str) -> list[Document]:
    """Load documents from a pickle file."""
    with open(pickle_filepath, "rb") as file:
        return pickle.load(file)


def initialize_vector_store(document_chunks: list[Document]) -> Chroma:
    """Reset the Chroma collection and initialize a vector store using document chunks."""
    Chroma().reset_collection()
    embedding_model = OpenAIEmbeddings(model=EMBEDDING_MODEL)
    return Chroma.from_documents(documents=document_chunks, embedding=embedding_model)

Let's load our documents and inspect the first one.

In [201]:
data_dir = "/content/data/"
data_file = "bloomberg_financial_news_1k.pkl"

# Load the Bloomberg documents from the pickle file
documents = load_documents(os.path.join(data_dir, data_file))

In [202]:
doc_str = f"{documents[0].metadata['Headline']}\n\n{documents[0].page_content}"
Markdown(doc_str)

Kuwait Wealth Fund Doubles Investments in U.K. to $24 Billion

Kuwait Investment Authority , the country’s sovereign wealth fund, doubled investments in the U.K. over the last 10 years to more than $24 billion. Investment in Britain was “across all asset classes, sectors and industries,” Managing Director Bader Al-Saad said in a speech in London on June 28. Kuwait Investment Office, the KIA’s London-based investment arm, now manages more than $120 billion globally compared with $27 billion 10 years ago. “Alongside the real estate subsidiary, which manages our real estate assets, we have set up an infrastructure fund which we are operating from London,” Al-Saad said. The investment office “will be looking to manage more funds, in more asset classes, and in more regions of the world in the near future,” he said. The KIA, which has about $342 billion of assets according to the Sovereign Wealth Fund Institute in California , holds stakes in Daimler AG and BP Plc. (BP/) The fund manages Kuwait’s General Reserve Fund and Future Generations Fund, and invests in local, Arab and international markets.

### Initializing the Vector Store and the Retriever

The vector store and retriever are key components of our RAG system. Here's what happens in this section:

- Initialize a new Chroma vector store with these documents
- Create a retriever that will fetch the `RETRIEVAL_K` most relevant documents according to their embedding

In [203]:
# Initialize the vector store with the documents
vector_store = initialize_vector_store(documents)

# Create a retriever instance from the vector store
retriever = vector_store.as_retriever(search_kwargs={"k": RETRIEVAL_K})

The retriever acts like a smart search engine - when given a question or topic, it returns the most relevant documents from our database. It does so by finding the documents similar embeddings to the query. In LangChain, this is also done with `.invoke()`. Let's try an example.

In [204]:
retrieval_query = "tech sector market trends"

# Invoke the retriever with the retrieval query
retrieved_documents = retriever.invoke(retrieval_query)

In [205]:
display(retrieved_documents)

[Document(id='a5f537ad-5e72-43cb-b998-855f8e64d591', metadata={'Headline': 'ARM Sees Shift to Cheaper Smartphones Boosting Chip Royalty', 'Link': 'http://www.bloomberg.com/news/2013-11-20/arm-sees-shift-toward-cheaper-smartphones-boosting-chip-royalty.html'}, page_content='ARM Holdings Plc (ARM) , the U.K. chip designer whose products power Apple Inc. (AAPL) ’s iPhones, predicts a shift in demand toward lower-priced smartphones will help boost royalty revenue. “There is a shift advantage for ARM as mid-range and entry smartphones drive smartphone growth. We’ve got that dynamic going into next year,” Chief Financial Officer Tim Score said at a conference in Barcelona yesterday. “Those phones generate much more royalty for ARM because they use more silicon.” ARM has had five consecutive years of revenue growth as its designs for chips that control graphics and processing were used in increasingly popular smartphones and tablets. Royalty payments took the spotlight last quarter as they mi

### Creating the RAG System

We can now augment our basic chatbot by providing it access to the retriever using **LangChain** tools, which allow the model to:
- Query the document database if needed
- Provide an answer based on the retrieved documents

#### Creating a tool with LangChain

We can create a tool using the `@tool` decorator from **LangChain** and provide it to the model using `.bind_tools()`. The model will receive all the relevant information about the tool thanks to the decorator. This way it knows how it works and can decide when to use it.

In [206]:
from langchain_core.tools import tool

In [207]:
@tool
def retrieval(retrieval_query: str) -> list[Document]:
    """Retrieve documents based on a query."""
    return retriever.invoke(retrieval_query)


# Create a list of tools and a dictionnary of tool functions by name
tools = [retrieval]
tools_by_name = {tool.name: tool for tool in tools}

In [208]:
RAG_PROMPT = """
You are a Financial Analyst with access to a Bloomberg Financial News database.

Start by identifying industry relevant to the sector you want to invest into. Think of upstream industries and downstream industries, eg if you want to invest in Generative AI think of upstream GPU manufacturer and downstream industries most impacted by GenAI for instance call centers.

Define an investment strategy for that sector.

Identify the key players and their challengers.

Create a table with an investment recommendation for each stock (buy, hold, sell) and the reason why rooted in the news extracted from the database. Quote the source and date of the news you refer to support the investment strategy for each stock. Present this in a table format.

Query the database to help the client with their request.

"""

# Create the RAG model by binding the base model with the retrieval tool
rag_model = base_model.bind_tools(tools)

In [209]:
request = "I want to invest in Generative AI"

# Message list for the RAG model
messages = [
    SystemMessage(RAG_PROMPT),
    HumanMessage(request),
]

# Invoke the RAG model with the messages
rag_response = rag_model.invoke(messages)

Let's check the answer. As we can see its content is empty, but a tool call has been made.

In [210]:
Markdown(f"Content: {rag_response.content}\n\nTool Calls: {rag_response.tool_calls}")

Content: To invest in Generative AI, we need to consider both upstream and downstream industries. 

### Upstream Industries:
1. **GPU Manufacturers**: Companies that produce graphics processing units (GPUs) are critical for the computational power required for Generative AI. Key players include:
   - NVIDIA
   - AMD (Advanced Micro Devices)
   - Intel

2. **Cloud Service Providers**: Companies that provide cloud computing services, which are essential for hosting AI models and applications. Key players include:
   - Amazon Web Services (AWS)
   - Microsoft Azure
   - Google Cloud

### Downstream Industries:
1. **Software Development**: Companies that develop applications utilizing Generative AI, such as:
   - OpenAI
   - Anthropic
   - Stability AI

2. **Customer Service**: Industries that are heavily impacted by Generative AI, such as call centers and customer support platforms.

### Investment Strategy:
1. **Diversification**: Invest in both upstream and downstream players to mitigate risks.
2. **Focus on Growth**: Target companies with strong growth potential in the Generative AI space.
3. **Monitor Trends**: Keep an eye on technological advancements and regulatory changes that could impact the sector.

### Key Players and Their Challengers:
- **NVIDIA**: Leader in GPU technology but faces competition from AMD and Intel.
- **OpenAI**: A frontrunner in Generative AI applications, competing with Anthropic and Stability AI.
- **Amazon AWS**: Dominates cloud services but competes with Microsoft Azure and Google Cloud.

Now, I will query the Bloomberg Financial News database to gather recent news articles that can help inform our investment recommendations for these companies.

Tool Calls: [{'name': 'retrieval', 'args': {'retrieval_query': 'NVIDIA Generative AI news'}, 'id': 'call_wKjqHY6l9ft1H74dk0mlBQFg', 'type': 'tool_call'}, {'name': 'retrieval', 'args': {'retrieval_query': 'AMD Generative AI news'}, 'id': 'call_u3u6Hl1CNl3RLL3U1RfILeP3', 'type': 'tool_call'}, {'name': 'retrieval', 'args': {'retrieval_query': 'Intel Generative AI news'}, 'id': 'call_Kfu3RJET995xEQmpYxizqgoZ', 'type': 'tool_call'}, {'name': 'retrieval', 'args': {'retrieval_query': 'OpenAI news'}, 'id': 'call_dnQXVbmA271Fsy6S3GJBjS0f', 'type': 'tool_call'}, {'name': 'retrieval', 'args': {'retrieval_query': 'Amazon AWS Generative AI news'}, 'id': 'call_n3tup0QumrOoun8UtpI0CuAX', 'type': 'tool_call'}, {'name': 'retrieval', 'args': {'retrieval_query': 'Microsoft Azure Generative AI news'}, 'id': 'call_PwsOXsyJHkMBlhF7IPCwLtQN', 'type': 'tool_call'}, {'name': 'retrieval', 'args': {'retrieval_query': 'Google Cloud Generative AI news'}, 'id': 'call_0xT5XEjRATftNZf5mHD11ZfA', 'type': 'tool_call'}]

Let's use the retrieval tool to retrieve documents following the model's query.

In [211]:
# Check if the RAG model response contains tool calls
if rag_response.tool_calls:
    # Get the first tool call from the response
    tool_call = rag_response.tool_calls[0]

    # Get the tool from the tool call
    tool = tools_by_name[tool_call["name"]]

    # Invoke the tool with the tool call arguments
    documents = tool.invoke(tool_call["args"])

    # Combine the retrieved documents into a single string
    documents_str = "\n\n".join(
        [f"{doc.metadata['Headline']}\n\n{doc.page_content}\n" for doc in documents]
    )

In [212]:
Markdown(documents_str)

Nissan Opens Silicon Valley Center for Self-Driving Car Research

Nissan Motor Co. opened a research center in California’s Silicon Valley that the Japanese carmaker plans to make its hub for research on self-driving vehicles and Internet-connected auto technology. The facility will be staffed by more than 60 engineers and technicians within three years, Carla Bailo , Nissan’s senior vice president for North American research and development, said in a phone interview. Work on so-called autonomous vehicle systems will move from an R&D center in Japan to Sunnyvale, she said, declining to provide investment details for the project. “We’re going to focus on this technology really in the heart of where it lies today,” Bailo said in the Feb. 15 interview. “We are going to be shifting work that’s being done in our Nissan Advanced Technical Center in Atsugi, and moving that work over to the heart of the industry.” Automakers and companies including Google Inc. plan new electronics and programming for vehicles with a goal of reducing collisions, fuel use and improving traffic flow. Carlos Ghosn , Nissan’s chief executive officer, told reporters last month in Detroit that regulatory, legal and technological hurdles mean self-driving cars won’t be feasible until about 2020. Google has tested a self-driving Prius in California for years, and the U.S. Defense Department has sponsored autonomous- vehicle research for more than a decade. Nissan’s Sunnyvale center will work with automotive labs at Stanford University, the University of California’s Berkeley and Davis campuses, and Silicon Valley companies, Bailo said. The Yokohama-based carmaker plans to apply technology developed at the new R&D center within a decade, she said. Maarten Sierhuis , a former NASA scientist specializing in artificial intelligence research, will lead the Sunnyvale facility, Nissan said today. Toyota Motor Corp. opened its own Silicon Valley outpost in Mountain View in 2012 to work with technology companies there on in-car data and entertainment systems. The company said in January that it’s developing autonomous safety systems to create a virtual “co-pilot” in vehicles to help drivers avoid accidents rather than self-driving cars and trucks.


Nicolas Cage Hired by China’s BAIC Motor to Promote Senova Cars

Actor Nicolas Cage has signed on as brand ambassador for Chinese automaker BAIC Motor Corp., joining Brad Pitt and Barcelona soccer star Lionel Messi in promoting automobiles in the world’s largest vehicle market. Cage will appear in a television commercial to be aired in May, according to Beijing-based BAIC Motor. The state-owned automaker said it chose the 49-year-old Academy Award winner as its brand spokesman because he’s well-known to Chinese audiences, especially target buyers in their 40s. “Cage will be our ambassador for the Senova D series that will hit the market soon,” Alex Qu, a spokesman for the company, said by phone today. Qu declined to provide more details on the events that Cage will attend in China. BAIC, whose parent traces its roots to making sidecar motorcycles used in the Korean War , is stepping up publicity as it prepares to roll out its Senova brand in the second quarter. The automaker is also targeting sales to senior government officials, with Beijing municipal Party Secretary Guo Jinlong and Mayor Wang Anshun expressing interesting in replacing their Audi sedans, BAIC’s overseas business head Dong Haiyang said. Cage, currently the voice of a caveman in the animated film “The Croods,” won an Oscar for his performance in “Leaving Las Vegas ” and has also starred in movies such as “Drive Angry.” General Motors Co. (GM) signed up Pitt in January to promote its China-made Cadillac XTS, while Chery Automobile Co. hired Barcelona soccer player Messi in 2010 to pitch their Riich brand cars. Zhejiang Geely Holding Group Co.’s Volvo Cars entered an endorsement deal with Houston Rockets player Jeremy Lin last year. BAIC Motor’s parent has said it is seeking an initial share sale for the car unit in Hong Kong and mainland China. To contact Bloomberg News staff for this story: Tian Ying in Beijing at  ytian@bloomberg.net 


Fusion-io Surges on Sales Forecast and Acquisition of NexGen

Fusion-io Inc. (FIO) , a maker of data- storage computers, climbed the most in eight months after forecasting sales that topped analysts’ estimates and announcing the acquisition of NexGen Storage Inc. Revenue in the fourth quarter, which ends in June, will be about $110 million, the Salt Lake City-based company said yesterday in a statement. That’s higher than the average analyst estimate of $107.5 million, according to data compiled by Bloomberg. Fusion-io, whose biggest customers include Apple Inc. (AAPL) and Facebook Inc., is aiming to sell to small and medium-sized businesses with more affordable flash-memory-based services. The company said it purchased NexGen for about $119 million in cash and stock, and is already promoting NexGen’s hybrid storage product on its website. Fusion-io rose 20 percent to $20.03 at 9:46 a.m. in New York, and earlier touched $20.20 for the biggest intraday gain since August. The stock had dropped 27 percent this year through yesterday, compared with an 11 percent gain for the Standard & Poor’s 500 Index. While revenue in the fiscal third quarter fell 7 percent to $87.7 million, it topped the $80.4 million average analyst estimate. Fusion-io said its net loss widened to $20 million, or 21 cents a share, from $4.68 million, or 5 cents, a year earlier. Fusion-io, which employs Apple co-founder Steve Wozniak as chief scientist, sells corporate storage computers based on flash memory , which is faster and more expensive than widely used magnetic disk drives. NexGen, based in Louisville, Colorado , introduced its product in November 2011, a week after saying it raised $10 million from Next World Capital, Grotech Ventures and Access Venture Partners. “This would take years to build out as a stand-alone company,” NexGen Chief Executive Officer John Spiers wrote yesterday in a blog post. “Overnight, those concerns are eliminated.”


Daimler to Add Renault-Nissan Fuso Vans to Global Cooperation

Daimler AG (DAI) and Renault SA (RNO) are expanding their global cooperation to include commercial vans made in Japan as the manufacturers continue to identify new projects together. Renault’s alliance partner Nissan Motor Corp. (7201) will supply Daimler with the NV350 Urvan to be sold under its Mitsubishi Fuso brand in selected export markets, the companies said today at the International Auto Show in Frankfurt. “We will continue to seek out new projects together that make sense for all partners,” Daimler Chief Executive Officer Dieter Zetsche said at a news conference. The manufacturers have gradually added projects to their three-year-old cooperation, which was originally focused on Stuttgart, Germany-based Daimler’s Smart city cars and Mercedes-Benz delivery vans. Last year, the companies agreed to jointly develop a four-cylinder gasoline engine and to license Mercedes transmissions for Nissan’s Infiniti luxury brand to be made in Mexico. “There’s no limit to the cooperation, there’s no taboo,” Carlos Ghosn , chief executive officer at both Renault and Nissan, said at the event. Tennessee Production The ramp up of a joint engine production in Decherd, Tennessee is on track for production start in mid-2014, Ghosn said. The engines will be used in the mid-sized Mercedes-Benz C-Class and new Nissan Infiniti cars. Mercedes and Infiniti are not competing for the same customers, said Ghosn. “Cross-shopping between the two brands is limited.” The two partners will also look into a possible cooperation in the large van segment even as talks haven’t started yet, Zetsche said. A Daimler contract to rebadge the Mercedes Sprinter for Volkswagen AG (VOW) will cease at the end of 2016, Daimler said last week. The decision to end production for Volkswagen was independent of any possible cooperation with Renault-Nissan, Zetsche said. Daimler and Renault, which is based in the Paris suburb of Boulogne-Billancourt, cooperate on Daimler’s smaller Citan delivery van, which the German company adapted from Renault’s Kangoo. The manufacturers are also considering teaming up on future compact car models. The review continues and no decision on a possible joint production in Mexico has been made, Ghosn said.


GT Advanced to Idle Plant, Expects Solar Recovery in 2014

GT Advanced Technologies Inc. (GTAT) , a U.S. supplier of manufacturing equipment for solar and energy- efficient lighting products, plans to sell a factory in St. Louis. The facility is developing high-efficiency solar technology that may not be needed until demand for photovoltaic panels picks up next year. The company will fire 35 people at the polysilicon plant, according to a statement today. Nashua, New Hampshire-based GT Advanced also said it will scale back its acquisition efforts this year. A global glut of solar production capacity drove down panel prices 61 percent in the past two years and slowed sales of GT Advanced’s systems. The company is transferring development of a high-efficiency polysilicon technology, HiCz, to a Merrimack, New Hampshire , plant from St. Louis. The systems will be ready this year and Chief Executive Officer Tom Gutierrez expects orders in 2014 when he anticipates solar demand to recover. “Given current market conditions in the PV industry, orders for the new generation systems may not occur until 2014 when the industry begins to recover,” the company said in the statement. Idling the plant will save about $15 million annually. The company will incur charges, which weren’t quantified, for the fourth quarter and in the current year for the restructuring. 2013 Forecast GT Advanced expects 2013 earnings of 25 cents to 45 cents a share, it said Dec. 18. That forecast includes the anticipated savings from the St. Louis plant, according to today’s statement. GT Advanced also expects to take a charge of $80 million to $90 million for the fourth quarter to write down the majority of its inventory of its existing line of polysilicon production systems, known as the DSS family. David Gray , chief strategy and new business officer, will leave the company Jan. 25 and his position is being eliminated as Gutierrez assumes responsibility for GT Advanced’s reduced acquisition efforts The new structure will “streamline our operations and align our business with current market and business conditions,” Gutierrez said in the statement. GT Advanced fell 0.9 percent to $3.41 at the close in New York.


Brain Made Transparent May Lead to Clearer Understanding

Scientists have developed a technique to make brains transparent, enabling them to see vast networks of neurons and structures for a big picture view of the organ that’s mostly studied in slices. In a report published today in the journal Nature, researchers at Stanford University described the method that replaced fats in the brain of mice with a gel that made the organ transparent. The scientists were able to make images of the brain’s structures and see down to the cells and molecules, according to the paper. The approach also was used on a human brain, the report said. Earlier this month, President Barack Obama announced the BRAIN Initiative, which will spend $100 million beginning in 2014 to map the complex interactions between neurons. The new technique may aid the research, allowing long-range circuits to be more easily mapped, said Michelle Freund, of the National Institute of Mental Health. “The BRAIN Initiative recently announced by the White House is designed to facilitate innovative strategies to better understand the brain,” Freund said in a video released by the institute. “In order to do that, we need new technologies, and clarity is a great example of the type of technologies that will enable future brain studies.” Previous methods required scientists to slice brains into small sections to look at anatomy. This slicing deformed the organs and made it difficult for researchers to see large-scale information running across the slices. Fat Replacement The researchers at Stanford near Palo Alto, California , replaced the fats that support the brain’s cells with a clear material that held all the delicate systems in place. First, the brains were soaked in a chemical bath and then heated, creating a clear gel in place of the fats. That allowed the scientists more access to the whole brains. The team tested the technique, dubbed Clarity, on the brain of a mouse that had been genetically tweaked to produce a fluorescent protein. Under an electron microscope, the scientists were able to see even the spaces between the neurons. What’s more, the researchers tested Clarity on the brain of an autistic person that had been preserved in formaldehyde for six years. There too, scientists saw individual nerve fibers. Since many brains have been preserved in formaldehyde for years, those organs stored in libraries may also be studied.


ARM Sees Shift to Cheaper Smartphones Boosting Chip Royalty

ARM Holdings Plc (ARM) , the U.K. chip designer whose products power Apple Inc. (AAPL) ’s iPhones, predicts a shift in demand toward lower-priced smartphones will help boost royalty revenue. “There is a shift advantage for ARM as mid-range and entry smartphones drive smartphone growth. We’ve got that dynamic going into next year,” Chief Financial Officer Tim Score said at a conference in Barcelona yesterday. “Those phones generate much more royalty for ARM because they use more silicon.” ARM has had five consecutive years of revenue growth as its designs for chips that control graphics and processing were used in increasingly popular smartphones and tablets. Royalty payments took the spotlight last quarter as they missed analysts’ estimates. ARM shares have fallen almost 9 percent since the earnings report on Oct. 22. “As ARM technology becomes more sophisticated, the royalty percentage goes up,” Score said. “On average, it’s going to very gradually go up.” Global smartphone shipments surged 40 percent last quarter, slowing from 53 percent growth for the previous three-month period, according to researcher IDC. The average selling prices of the devices fell by 13 percent to $317 from a year earlier, because of demand for more affordable handsets. In spite of concerns about slowing growth, especially in China, Score said ARM can continue expanding its customer base and outperforming the semiconductor market in the next five years, driven by cheaper handsets, enterprise networking and micro-controllers. ARM shares fell less than 1 percent to 953.50 pence at 9:16 a.m. in London. New Chief Chief Executive Officer Simon Segars, who succeeded Warren East in July, is confronting a shift in the industry’s growth to low-end smartphones and new entrant Intel Corp. (INTC) , which has begun designing its own chips for mobile phones. ARM, based in Cambridge, England, is also seeking to boost its market share in servers as companies from Facebook Inc. (FB) to Google Inc. (GOOG) look for lower-power machines to drive their data centers. It’s “possible to identify such a market and build a chip specifically for that market, whether it is to handle only storage or only video loading, for example. We can make the chip much more efficient,” Score said at the annual Technology, Media and Telecoms Conference organized by Morgan Stanley. “Everyone knows Intel has a very strong ecosystem around servers, we’re not kidding ourselves,” Score said. “For that reason we see it as further out.”


SolarCity Rises on Credit Suisse Upgrade: San Francisco Mover

SolarCity Corp. (SCTY) , the owner of rooftop solar plants led by billionaire Elon Musk, climbed the most in three weeks after analysts at Credit Suisse Group AG upgraded shares and almost doubled their 12-month price target. SolarCity rose 6.6 percent to $36.29 at the close in New York, the most since May 20. The San Mateo, California-based company’s shares began trading Dec. 13 at $8. Demand for rooftop power systems in California , the largest solar market, will grow, buoyed by support from state regulators and officials, according to Credit Suisse analysts Satya Kumar, Brandon Heiken and Patrick Jobin. Governor Jerry Brown has set a goal of 12 gigawatts of so-called distributed generation by 2020, up from about 1.7 gigawatts today. Brown “has been a strong proponent of distributed generation, countering utilities’ opposition,” the analysts said in a research note today. They raised their rating to the equivalent of buy and increased SolarCity’s 12-month price target to $52 from $28. Distributed-generation systems typically aren’t owned by utilities and produce power close to where it’s consumed by homeowners and businesses. Utilities are required under state regulations to purchase at retail rates any surplus electricity that’s shipped to the grid. The regulation, known as net metering, has a 5 gigawatt cap and may expire in 2015. “We are more comfortable that the CPUC and California governor will continue to support distributed solar generation even if the net metering cap is not raised by 2015,” the analysts said. The California Public Utilities Commission is appointed by Brown. Billionaire Musk, 41, is SolarCity’s chairman and largest shareholder with about 28 percent. He is also chief executive officer of Tesla Motors Inc. (TSLA) , the electric car company that’s more than doubled in value this year. SolarCity CEO Lyndon Rive , Musk’s cousin, holds about 3.5 percent of his company.


KDDI to Sell $100 Million Gree Stake to Book Investment Profit

KDDI Corp. , Japan’s second-biggest wireless carrier, plans to sell a $100 million stake in mobile- game operator Gree Inc. to book a profit from its six-year investment. The carrier has asked Daiwa Securities Group Inc. to sell 8 million Gree shares between Feb. 13 and May 12, it said in a regulatory filing today. KDDI, the second-largest shareholder in Gree , will retain 8 million shares, or about 3 percent. “We can profit from the sale,” Keiichi Sakurai, a spokesman for Tokyo-based KDDI, said by phone today. “The sale will also let Gree do business more freely.” Gree has more than doubled in Tokyo trading since listing in 2008 on rising mobile phone usage and the popularity of games including “Monster Quest” and “Crime City”. This year, the company has fallen 14 percent after forecasting a decline in annual profit because of delays to new games and costs from expanding overseas. The game developer rose 2.7 percent to 1,154 yen in Tokyo trading today before the KDDI announcement. KDDI first invested in the company in 2006, according to Gree’s website. “There won’t be any change in our relationship,” Shinichi Iriyama , a Gree spokesman, said by phone. “We will continue building a good relationship with KDDI.”


Pacific Ethanol to Use Edeniq Milling Technology to Boost Output

Pacific Ethanol Inc. (PEIX) , a U.S. biofuel producer, is seeking to increase production at a California plant using technology from Edeniq Inc. Pacific Ethanol expects to boost yields by 2 percent to 4 percent using Edeniq’s milling system, which converts corn and other plant feedstock materials into particles that are processed into fuel more efficiently, the Sacramento, California-based company said today in a statement. The company will install the so-called Cellunator technology this quarter at its plant in Stockton, California. It will also use another Edeniq system that increases corn-oil recovery, according to the statement. Pacific Ethanol has four plants in California, Oregon and Idaho that produce a total of 200 million gallons (758 million liters) of fuel a year. The Stockton site’s annual capacity is 60 million gallons. Edeniq, based in Visalia, California, received a $3.9 million grant in June from the California Energy Commission to develop cellulosic ethanol production technology, according to the statement.


Nissan CEO Wants at Least 20% Leaf Gain With Cheaper Car

Carlos Ghosn , who leads Nissan Motor Co. (7201) and Renault SA (RNO) , said he expects global deliveries of all- electric Nissan Leafs to expand at least 20 percent this year aided by the addition of a lower-priced version. The company today said the new S trim version of its rechargeable hatchback will have a $28,800 base price when it goes on sale this year. Prices for its current SV and SL grade cars for 2013 will be $31,820 and $34,840, respectively, the company said at the North American International Auto Show in Detroit. Nissan, Japan’s second-largest carmaker, sold 9,819 Leafs in the U.S. last year, less than half of Ghosn’s 20,000 goal. Ghosn cited limited charge infrastructure as one reason for the result and declined to set a specific sales target for the lithium-ion powered hatchback “You can expect them to be up,” Ghosn told reporters today in Detroit. “It would be fair to say” that Leaf should post at least a 20 increase, he said. Base prices for the 2012 SV and SL Leafs are $35,200 and $37,250, respectively, before a $7,500 federal tax credit, according to Nissan’s website. He said the company’s bullish early targets for the car reflected “passion” for the product rather than “arrogance.” Production of the car and its battery packs is under way in Smyrna, Tennessee, the company said this month. Nissan is coming off record U.S. sales in 2012, aided by deliveries of Altima sedans and Rogue crossovers. The Yokohama, Japan-based company boosted deliveries 9.5 percent to 1.14 million Nissan and Infiniti brand vehicles. Nissan’s U.S. headquarters are in Franklin, Tennessee.


Murata to Double Sensor Sales on Smartphones, Auto Demand

Murata Manufacturing Co. , a Japanese supplier to Apple Inc. and Samsung Electronics Co., expects to double sensor sales in four years because of demand for more sophisticated cars and smartphones. Sensor revenue will probably jump to about 75 billion yen ($806 million) in the year ending March 2016 from less than 40 billion yen last fiscal year, Vice President Satoshi Sonoda said in an interview yesterday at the company’s headquarters in Kyoto, Japan. The devices, which detect movement or pressure on devices and then generate an electronic signal, support features such as touchscreen mobile phones and in-car navigation systems. Murata expects total electronic-component sales to carmakers to rise about 10 percent a year, about double the pace of growth in the global vehicle market, because of tighter safety and environmental standards, Sonoda said. The company announced three acquisitions this month to expand its lineup and speed up development. The component maker will buy out Tokyo Denpa Co ., which makes quartz-based products used in smartphones, in a stock swap, the two companies said in a joint statement today. Murata will also offer 300 yen a share to raise its stake in Toko Inc., a maker of power inductors, to 66.6 percent through a tender offer, Murata said separately. Ultrasonic Sensor Founded in 1944, Murata is also developing a new ultrasonic sensor that can detect human movements such as waving without users’ touching displays, Sonoda said. The company aims to start mass production of the sensor by the end of the year at the earliest, he said. The company boosted its focus on automobiles last year by buying VTI Technologies Oy, a Finnish maker of vehicle sensors, for 20 billion yen, including debt. The component maker fell 1.8 percent to 5,880 yen on the Osaka Securities Exchange. It has gained 16 percent this year , compared with an 8.2 percent advance for the benchmark Nikkei 225 Stock Average. Sensors account for about 7 percent of Murata’s total revenue. The company’s other products include components for power, sound and RFID.


GM to Start Third Shift at Brazil Complex, Add 2,630 Positions

General Motors Co. (GM) said it will begin a third shift at the Gravatai Industrial Complex in Brazil and add 2,630 factory and supplier jobs. The Chevrolet Celta and Onix are made at the complex, according to a statement on GM’s website. The Detroit-based automaker will start production of another small sedan in February, prompting the third shift, Jaime Ardila, president of GM South America, said in the statement. GM didn’t provide additional details. “We launched nine all-new Chevrolet models in the last 15 months in the Brazilian market and are poised for growth,” Ardila said in the statement. GM will add 1,450 jobs at a vehicle-assembly plant and 180 at a powertrain factory, while suppliers will add 1,000 jobs, according to the statement. GM is looking to improve financial results in South America as the company’s growth in China slows and losses in Europe persist. Brazil accounted for 60 percent of the South American unit’s vehicle sales during the first nine months of 2012, according to a U.S. regulatory filing.


Oxford Nanopore’s Gene Sequencer Facing Technical Delays

Oxford Nanopore Technologies Ltd., the company in a race with Illumina Inc. (ILMN) and Life Technologies Corp. (LIFE) to develop smaller, cheaper and quicker gene-sequencing devices, is facing technical setbacks. Oxford Nanopore, based in Oxford, England , has had to redesign components of a custom sensor because it wasn’t performing satisfactorily, which has lengthened development and testing periods, according to a post yesterday on the blog Pathogens: Genes and Genomes. The blog is found on the University of Birmingham’s website and written by Nick Loman, a doctor who works as a bioinformatics specialist at the English university. “That put us back about five months, but it was the right thing to do,” Loman quoted Oxford Nanopore Chief Technology Officer Clive Brown as saying at the Advances in Genome Biology and Technology conference last month. The closely held company said more than a year ago that it expected to begin selling its products by the end of 2012. Yesterday, it didn’t deny Loman’s account of his conversation with Brown when contacted by Bloomberg News. “We should have made it clear that was off the record,” Chief Executive Officer Gordon Sanghera said of Loman’s report. “We aren’t giving any further comment on it.” The company has also had to address chemistry in its devices degrading and has been striving to reduce the error rate to 1 percent from 4 percent, according to Loman’s account of his conversation with Brown, which took place during the Feb. 20-23 AGBT meeting in Marco Island, Florida. Technical Risks IP Group Plc (IPO) owns about 20 percent of Oxford Nanopore. Competitor Illumina owns about 15 percent, and the company’s other shareholders include hedge-fund manager Lansdowne Partners LP and Invesco Perpetual, the U.K. group of mutual funds. Numis Securities’ London-based analyst Charles Weston reduced his estimate of Oxford Nanopore’s value by 25 percent to $1.5 billion from $2 billion in a Feb. 20 note to clients, citing technical-development risks. Loman wrote that Brown told him the device wouldn’t be “a year, two years late” and that the company would release data from its early-access program for its products this year. Oxford Nanopore was spun out of Oxford University in 2005 and its technology was initially based on research by Hagan Bayley, a professor of chemical biology at the university. Its instruments use a novel technique known as strand sequencing, in which an entire string of DNA is guided by an enzyme and passes intact through a tiny hole in a cell membrane one-billionth of a meter wide.


NBA Unveils Advanced Statistical Player Analysis on Its Website

The National Basketball Association added advanced statistical categories to its website , joining a movement toward increased use of sports data to help teams, fans and media members analyze player performance. The website now includes every box score in NBA history, shooting charts that help visualize offensive and defensive performances and statistics that break down player attributes such as efficiency and how clutch they are, the league said in a statement. “Sports fans have a passion for statistics because it brings them closer to the game, their favorite teams and players,” said Steve Peck , senior vice president of global strategic initiatives at SAP AG , which partnered with the NBA to power the website. The NBA’s entire official statistics history now will be offered free after previously being made available only to teams and league officials. Content found on the site can be shared on social media platforms Twitter, Facebook and Google+. Using advanced data to analyze player performance has become common in Major League Baseball since Bill James used statistics to help make the Boston Red Sox World Series champions in 2004. The movement was popularized by Michael Lewis ’s book “Moneyball,” which chronicled Oakland Athletics General Manager Billy Beane ’s use of statistics to build his roster. Lewis is a Bloomberg News columnist.


China State Grid Builds More Networks to Handle Wind Capacity

China State Grid Corp., the nation’s largest power distributor, is boosting efforts to build more electricity networks to enhance the grid’s ability to handle power from new sources such as wind. Preliminary work is accelerating on an ultra-high voltage link to deliver wind power from Jiuquan in the northwestern province of Gansu to southern Hunan, said Zhang Zhengling, deputy director of the development and planning department at China State Grid. Two grid lines are also being built to transmit power from northern Hebei to areas including Beijing and Tianjin city, he said in an interview yesterday. Grid improvements in China are necessary to deal with an influx of new energy sources. In particular, enhancements would help reduce idled wind capacity, particularly in the north. The state-owned grid operator is creating capacity in areas where the issue of idled wind power is most severe, Zhang said at a conference yesterday. The annual rate of China’s idled wind capacity may fall to 12 percent this year from 17 percent in 2012, Guo Yanheng, the deputy director of the National Renewable Energy Engineering Information Management Center, said in August. Turning wind farms off cost operators at least 10.6 billion yuan ($1.74 billion) in lost revenue last year, almost doubled 2011’s figure, according to the renewable energy center. China plans to take two to three years to resolve the issue, Shi Lishan, deputy director of the renewable energy department under the National Energy Administration, said. Wind Farms China has optimized the planning of wind farms, with new approvals centered in Shandong and Shanxi provinces and stricter controls in the north, Zhang said. The electricity lost because of wind curtailment, the term used to describe wind farms producing at less than their generation capacity, may fall at least 25 percent to below 15 billion kilowatt-hours this year from a year ago, he estimates. By the end of the year, the grid company will start operating another line from Hami in Xinjiang to Henan, which can handle 8 gigawatts of wind power and 1.25 gigawatts of solar, Zhang said at the conference. To contact Bloomberg News staff for this story: Feifei Shen in Beijing at  fshen11@bloomberg.net 


Detnor Gains After Raising Jette Oil Output Guidance: Oslo Mover

Det Norske Oljeselskap ASA (DETNOR) , an energy producer with stakes in Norway’s biggest find in 40 years, pared gains in Oslo trading after increasing production guidance for its Jette field in the North Sea by less than had been reported. The Trondheim-based company gained 1.9 percent to 92.5 kroner as of 1:07 p.m. local time, having earlier surged as much as 3.6 percent to 94 kroner, the highest intraday level in seven months. Detnor, which started output from Jette on May 21, expects gross initial production from the field’s two wells of about 12,000 to 15,000 barrels of oil a day, up from earlier guidance of about 10,000 barrels, it said in a statement. Shares jumped earlier after Offshore.no, an oil-industry website, said Jette is producing at about 21,000 barrels a day, citing an unidentified person. Output at that rate would have been double expectations, Alex Gheorghe, an analyst at RS Platou Markets AS, said in an e-mail. “The news is positive since the project was written off last year, and management has previously disclosed it’s unlikely it will recover costs,” Gheorghe wrote. “We expect the stock to outperform by about 1 to 2 kroner a share.” Detnor is transforming itself from an exploration company into an oil producer as pumping begins at fields including Jette and Ivar Aasen, scheduled to start in the fourth quarter of 2016. The company took a 1.9 billion kroner ($329 million) writedown on Jette in the third quarter after technical problems with a production well. “Jette is a small field and there are uncertainties with respect to future production rates,” Detnor said today. The company has a 70 percent stake in Jette, while Petoro AS, which manages Norway ’s direct stakes in its oil and gas assets, has the remaining 30 percent. Shares in Detnor have more than tripled in the last two years after finds including Johan Sverdrup, which could hold as much as 3.6 billion barrels of oil, making it the biggest discovery off Norway since 1974. Detnor has stakes in two of the three licenses containing Sverdrup.


Intel CEO Krzanich Plans Faster Shift to Mobile Chips

Intel Corp. (INTC) ’s Brian Krzanich, who took over today as chief executive officer, said the world’s largest computer chipmaker has the assets it needs to accelerate a push into mobile devices. “We see that we’ve been a bit slow to move into that space,” Krzanich said at the annual shareholder meeting at the company’s headquarters in Santa Clara , California. “The base of assets that we have will allow us to grow in that area much faster moving forwards.” Krzanich, Intel’s sixth chief executive, takes the helm as the company struggles to cut into Qualcomm Inc. (QCOM) ’s lead in the $85.4 billion mobile-chip market. Industry-leading manufacturing and new products will enable the company to start making inroads in phones and tablets, he said. Krzanich said he and newly promoted company President Renee James will try to make Intel more outward-looking in order to predict future trends in computing, Krzanich said. That’s more important as consumers, rather than corporations, take the leading role in determining which devices and technologies are successful, Chief Financial Officer Stacy Smith said in an interview following today’s meeting. “The lifecycle of those devices and the lifecycle of the leadership of the market is very short,” he said. “The consumer is driving these very fast technology shifts.” Krzanich, a former factory manager, succeeds Paul Otellini , who retired. Intel has improved its execution in creating products for new markets and Krzanich, with his background in operations, is very focused on making sure that there are no slip ups, said Smith. Intel shares rose less than 1 percent, to $24.23 at 2:05 p.m. in New York. While the stock has gained 18 percent this year, it dropped more than 6 percent during Otellini’s tenure, which began in May 2005.


Chrysler U.S. Sales Chief Sees Growth Slowing in 2013

The U.S. auto industry’s three-year streak of at least 10 percent growth probably will end this year, said Reid Bigland , head of sales in the market for Chrysler Group LLC. “I don’t see it increasing by double digits into 2013,” Bigland said of the U.S. light-vehicle market today at the North American International Auto Show in Detroit. Credit availability, pent-up demand and an improving U.S. economy are reasons “still in place” to drive a sales increase this year, he said. Toyota (7203) Motor Corp., Honda Motor Co. and Chrysler last year led U.S. vehicle sales to their highest annual total in half a decade as the two Japan-based automakers restored production that was disrupted a year earlier by Japan ’s tsunami. Deliveries of cars and light trucks rose 13 percent, the biggest increase since 1984, to 14.5 million. U.S. auto sales may climb to 15.1 million this year, the average of 18 estimates in a Bloomberg survey of analysts this month. The industry is benefiting from consumers replacing cars and trucks that are the oldest ever on U.S. roads, at almost 11 years on average, according to researchers R.L. Polk & Co. and Experian Automotive. “The average age of the fleet on U.S. roads of a car is right around 11 years -- they don’t last forever,” said Bigland, 45, who declined to give a specific forecast for 2013 U.S. industry sales. The Federal Reserve ’s quantitative easing program has led to “very competitive interest rates ,” he said. Vanity, Sanity Growth this year probably will be led by Volkswagen AG (VOW) , Honda and Nissan Motor Co., according to five analysts surveyed by Bloomberg about market share this month. The analysts’ average estimates are for little change to market share for Auburn Hills , Michigan-based Chrysler, General Motors Co. (GM) and Ford Motor Co. (F)  Of the six largest automakers by sales in the U.S., Chrysler was the only company to surprise industry analysts last year, gaining 0.7 percentage point to 11.4 percent market share. Analysts last year projected on average that the automaker’s share would slip 0.2 percentage point. Chrysler is majority owned by Turin, Italy-based Fiat SpA. (F)  Chrysler won’t “chase” market share this year with incentives or by boosting deliveries to fleet customers, said Bigland, who declined to predict the company’s market share this year. “Market share is vanity and profitability is sanity,” he said. “We’ve learned from our past and going out and chasing sales at all cost and market share at all cost. We very much want to run and operate in balanced, disciplined and methodical approach.”


Karuturi to Borrow From Sovereign Fund After First Ethiopia Crop

Karuturi Global Ltd. (KARG) , the world’s largest rose grower, said it will borrow more than $100 million from a sovereign wealth fund to invest in farming in East Africa after selling its first produce from a plantation in Ethiopia. The company, based in Bangalore, India , produced 21,000 metric tons of corn in the last quarter of 2012 that sold for about $6.5 million in Ethiopia, Managing Director Sai Ramakrishna Karuturi said in a phone interview on March 27. The harvest, grown on its plantation in western Ethiopia, showed the project was “not a disaster” after floods destroyed a 60,000- ton corn crop in September 2011, he said. The deal with the unidentified fund comes after development banks declined to provide assistance because of “unfair” criticism of commercial farming by advocacy groups, Karaturi said. “Hundreds of millions” of dollars will be advanced to the company by the end of April and invested in Ethiopian projects as well as Kenyan flower farms, he said. Ethiopia’s land-commercialization drive has been attacked by groups including the California-based Oakland Institute, which says the farms forcibly displace people and will increase poverty. Karuturi leased 100,000 hectares (247,105 acres) in the Gambella region from Ethiopia’s government in November 2010 for 2 million Ethiopian birr ($108,151) a year Horizon Plantations, majority owned by Saudi billionaire Mohamed al-Amoudi, leased a 20,000-hectare plot in the western Benishangul-Gumuz region in September to grow groundnuts to produce cooking oil. Jobs, Foreign-Exchange The benefits of commercial farming in lowland areas of the country include a boost to crop production to reduce hunger and more jobs and foreign earnings, according to Ethiopia’s Agriculture Ministry. Since September 2009, the central government has leased 416,803 hectares to 34 domestic and foreign investors with an average plot size of 12,259 hectares, data from the ministry’s website shows. The state identified 3.3 million hectares of land nationwide, an area about the size of Belgium , as suitable for large-scale farming. Karuturi’s contract stipulates it should develop half its plot within 12 months and the remainder the following year. The agreement starts from the date of signing or “from the date of receipt of all the clearances from the government.” The project to grow and process crops including cereals, sugar, palm oil and vegetables has “good support” from senior Ethiopian officials, Karuturi said. The leasing of an additional 200,000 hectares to the company is conditional on completion of the current phase, according to the contract. All land in Ethiopia is owned by the state.


EMGS Confident of Boosting Backlog After Signing Brazil Deal

Electromagnetic GeoServices ASA (EMGS) , a Norwegian oil and gas surveyor, said it’s confident of increasing its order backlog after signing an agreement that expands its coverage of the waters off Brazil. “Towards the end of the year we will have at least one longer-term contract communicated,” Chief Financial Officer Svein Knudsen said in an interview in Oslo today. “We’re promising improved backlog and we’re sticking to that promise.” EMGS, based in Trondheim, plans to almost double its order backlog to at least $100 million by the end of the first half, Knudsen said in a presentation today. Shares in the company have fallen more than 30 percent since the start of the year amid concerns that the company’s backlog is shrinking. That’s even after EMGS signed a three-year agreement with Royal Dutch Shell Plc for the provision of 3D electromagnetic services globally. EMGS, which uses electromagnetic technology to search for oil and gas under the seabed, signed a deal with Spectrum ASA (SPU) to access its multi-client 2D survey library in order to plan a multi-client 3D survey in the Foz de Amazonas basin off Brazil, it said in a statement today. The survey will cover blocks that are included in the 11th licensing round, it said. EMGS expects to agree on similar collaborations elsewhere in the world with potential partners including Petroleum Geo- Services ASA (PGS) and TGS Nopec Geophysical ASA, Norway ’s two-largest seismic companies, Knudsen said. EMGS uses a method of mapping the seabed that competes with the seismic technology employed by companies such as Schlumberger Ltd., TGS and PGS. Shares in EMGS gained as much as 3.3 percent and closed unchanged at 9 kroner in Oslo.


Dynegy Expects 2013 Loss of Up to $332 Million

Dynegy Inc. (DYN) , the U.S. independent power producer that emerged from bankruptcy protection last year, forecast a 2013 net loss of as much as $332 million as it refinances debt to increase cash. Adjusted earnings before interest, tax, depreciation and amortization will be $250 million to $275 million, according to a slide presentation filed with the U.S. Securities and Exchange Commission today before an analyst meeting in New York. The forecast is below the $305.5 million average of four analysts’ estimates compiled by Bloomberg. “The range for 2013 was slightly below our expectations,” Jon Cohen, a New York-based analyst for International Strategy & Investment Group, said today in an e-mail. Cohen rates the shares cautious, equivalent to a sell. Dynegy, the owner of natural-gas and coal-fired power plants, put a group of units into bankruptcy in November 2011 after electricity prices tumbled in the wake of the 2008 economic slump. The corporation sought bankruptcy protection in July 2012. Dynegy has three sell and three hold recommendations from analysts, according to data compiled by Bloomberg. The company is the third-largest U.S. independent power producer behind Calpine Corp. (CPN) and NRG Energy Inc. (NRG) Independent power producers sell their output on wholesale markets or under contract to utilities.


Mitsubishi Buys Kirin’s Food Flavoring Unit for $321 Million

Mitsubishi Corp. (8058) , Japan’s biggest trading house, will acquire Kirin Holdings Co. (2503) ’s domestic food flavoring unit for 30.5 billion yen ($321 million) to expand in consumer businesses. The trader agreed to buy 81 percent of Kirin Kyowa Foods Co. in June and the rest in January 2015, Tokyo-based Mitsubishi said in a statement. The deal will expand the range of businesses at Mitsubishi’s Life Sciences division. The unit, formed two years ago, focuses on seasoning, sweeteners and fermentation technology, the company said in the statement. Kirin Kyowa is at least the third food investment by Mitsubishi in as many months as the trader leads domestic rivals including Mitsui & Co. (8031) in shifting focus to health care, utilities and nutrition to counter lower profits from selling bulk commodities such as coal and iron ore. Consumer goods rose to 12.4 percent of Mitsubishi’s profit last year, from 9.8 percent in the previous period. “The global market for food ingredients and flavoring materials is said to be valued at $120 billion,” Mitsubishi said in the statement. Buoyed by Asia’s emerging economies that are improving living standards and putting pressure on food supply , there is an “increased demand for food products that are delicious and healthy from safe and secure sources.” Taking over Kirin Kyowa from the Japanese beverages group will give Mitsubishi a company that supplies large-scale food processors and retail food chains with flavor enhancers such as yeast extracts, nucleotides and hydrolyzed proteins. The food sciences company has operations in Japan, Indonesia and China, according to the statement. Mitsubishi, which owns 32 percent of Lawson Inc. (2651) , Japan’s second-largest convenience store operator, has in the last two years invested in meat and livestock in China with Cofco Corp., entered the salmon-farming business in Chile and added grain collection and coffee plantations in Brazil. The trader in December said it agreed to invest in a 3 billion-yen seafood venture in Thailand. In January, it made an offer to buy Japanese meat products retailer Yonekyu Corp. (2290) 


Linde Profit Misses Estimates as European Steel Demand Slows

Linde AG (LIN) , tied with Air Liquide SA (AI) as the world’s biggest industrial gas company by sales, reported fourth-quarter profit that missed analyst estimates as slowing demand in Europe stunted growth. Earnings before interest, tax, depreciation and amortization rose 14 percent to 967 million euros ($1.3 billion), the Munich-based company said today in a statement. Analysts had predicted 976 million euros. Linde proposed an annual dividend of 2.70 euros a share. Linde has doubled its North America gases sales with the $3.8 billion purchase of Lincare in August and the acquisition of Air Products & Chemicals Inc. (APD) ’s home care business in January. Chief Executive Officer Wolfgang Reitzle said in October he will seek further savings in purchasing, information technology and the cylinder and liquefied gas supply chain. “We have been able to hold our own, although the climate has worsened in the course of the year,” Chief Executive Officer Wolfgang Reitzle said in the statement. “Even though conditions are unlikely to improve, we are sticking to our targets.” Rival Air Liquide said last month that fluctuating global market conditions may mean it reviews its target of boosting sales by 8 percent to 10 percent through 2015. The development of cheap shale gas in North America , and of coal in China , as well as an “upheaval” in nuclear energy, may force the Paris- based company to follow customers such as steelmakers and chemical producers that are shifting production to low-cost regions, its Chief Executive Officer Benoit Potier said. Linde’s quarterly sales increased 18 percent to 4.22 billion euros, in line with estimates. Net income gained 8.8 percent to 346 million euros, beating the 333 million-euro estimate of the Bloomberg survey. It reiterated a target of 4 billion euros in operating profit this year, while seeking between 750 million euros and 900 million euros in further savings by 2016. Return on capital employed will be at least 14 percent from that year, with Ebitda of at least 5 billion euros. Linde said today. The plan will be completed by the successor to CEO Reitzle, who said March 3 he will not seek to extend his contract beyond May 2014. The search for a successor has been whittled down to two external candidates, Supervisory Board Chairman Manfred Schneider said at the same time. An estimated 7.67 trillion cubic feet of U.S. natural gas was extracted using fracking in the U.S. last year. That’s 17 times more than a decade earlier, according to the Energy Information Administration, which forecast production will reach 13.6 trillion cubic feet in 2035, accounting for 49 percent of gas output.


Tesla Rises to Record on Hong Kong Orders: Mover

Tesla Motors Inc. (TSLA) rose to a record closing price today after the company said it has received hundreds of orders for its new Model S sedan in Hong Kong , enough to double the number of electric cars in the city. The shares gained 4.2 percent to $120.09 in New York. The stock has more than tripled this year, compared with a 15 percent increase for the Russell 1000 Index. “Our number is over all electric cars in Hong Kong combined,” Kenneth Lui, Hong Kong sales manager for the Palo Alto, California-based company, said in an interview yesterday. Prices of the vehicle, which goes on sale in the city this winter, will be disclosed in the next couple of months, he said. Tesla, headed by billionaire Chief Executive Officer Elon Musk , has forecast Model S sales will reach 21,000 units this year, with deliveries to Europe and Asia beginning in the second half. The Model S, which starts at $69,900 in the U.S., is Tesla’s second vehicle after the $109,000 Roadster. Hong Kong had 303 registered private electric cars at the end of April, according to the city’s Transport Department.


Bega Cheese Bid for Warrnambool Can Proceed, Says Regulator

Bega Cheese Ltd. (BGA) , the underbidder in the battle for Warrnambool Cheese & Butter Factory Co., won Australian regulator approval for its A$407 million ($396 million) offer for the diary producer. The approval makes Bega the only of three bidders for Warrnambool to be cleared, as Saputo Inc. (SAP) ’s A$449 proposal is being reviewed by Australia’s Foreign Investment Review Board and Murray Goulburn Cooperative Co. seeks regulatory clearance through the Australian Competition Tribunal. Bega is considering its options over improving its offer, Executive Chairman Barry Irvin said Oct. 21 in an interview. “A merged Bega and Warrnambool Cheese & Butter would continue to be constrained by other dairy manufacturers that they compete more closely with in the acquisition of raw milk, including Murray Goulburn,” Australian Competition and Consumer Commission Chairman Rod Sims said in the statement. Montreal-based Saputo won support from Warrnambool’s board with an improved offer of A$8 a share made last week, topping a A$7.50 per share cash proposal from Murray Goulburn Cooperative Co. and above Bega’s cash and shares offer. Bega closed yesterday in Sydney at A$4.39, making its offer of 1.2 shares and A$2 cash for each Warrnambool share worth A$7.27. Warrnambool fell 0.8 percent to A$8.29 at 10:25 a.m. in Sydney trading. Dairy Demand The contest for Warrnambool, which makes brands including Sungold milk and Great Ocean Road cheese, comes as bidders attempt to raise production and add export infrastructure amid rising demand in Asia. China , the world’s most populous nation, is setting records for milk-powder imports to fill its milk production deficit, while dairy demand is rising in India , Indonesia and Vietnam , the Victoria-state based company said in an Oct. 9 presentation. Kirin Holdings Co. (2503) , Japan’s largest beverage maker, bought a 9.99 percent stake in Warrnambool on Oct. 29, the company said in a statement. Kirin’s Australian unit Lion, which has a contract with Warrnambool to manufacture cheese for its Coon and Cracker Barrel brands, may be seeking to block a Saputo takeover, according to Canaccord Genuity Group Inc. Bega Cheese holds 18 percent of Warrnambool and Murray Goulburn holds 17 percent, according to data compiled by Bloomberg.


Valeo Stock Rises as Revenue Gain Sustains 2013 Forecast

Valeo SA (FR) , France ’s second-biggest auto-parts maker, rose the most in more than five weeks after reporting third-quarter sales growth in China and North America and signaling that the European car market is stabilizing. Valeo jumped as much as 3.3 percent, the biggest intraday increase since Sept. 10, and was trading up 2.6 percent at 69.50 euros at 10:15 a.m. in Paris. Volume was 51 percent of the three-month daily average after a little more than an hour of trading. The stock, which has been at the highest price since January 2000 for the past week, has gained 85 percent this year, valuing the manufacturer at 5.53 billion euros ($7.56 billion). The French manufacturer reiterated a full-year target yesterday of a “slight increase” in operating profit as a proportion of revenue, “assuming stabilized market conditions in Europe .” Industrywide car sales in the region are set to shrink for a sixth consecutive year following an 18-month recession in the euro region that ended in the second quarter. Third-quarter sales at Paris-based Valeo increased 2.2 percent from a year earlier to 2.91 billion euros, the manufacturer said yesterday in a statement. Excluding acquisitions, disposals and currency effects, sales to carmakers rose 25 percent in North America , 13 percent in Asia and 10 percent in both Europe and South America , exceeding competitors’ growth in all markets, Valeo said. ‘Fastest Growing’ “This leaves Valeo the fastest-growing European auto supplier and highlights its credentials as an attractive growth stock,” Philip Watkins, a London-based analyst at Citigroup Inc., said today in a report to clients. Valeo, whose products span windshield wipers, headlights and stop-start ignition systems, is focusing on technology promoting vehicle safety, comfort and pollution reduction to increase profitability. Chief Executive Officer Jacques Aschenbroich pledged in March 2011 to boost annual revenue to 14 billion euros by 2015, propelled by sales of fuel-saving components. The third-quarter figures “reflect the gradual entry into production of the high order intake recorded by the group over the last three years and the strength of Valeo’s growth model,” with gains in both original-equipment and spare-parts sales demonstrating “balanced growth,” Aschenbroich said in the statement yesterday. Valeo is sticking to predictions that Europe’s car market will shrink 2 percent to 3 percent this year, with worldwide auto sales rising 2 percent, Aschenbroich told journalists and analysts on a conference call yesterday.


GE Sells $895 Million of Wind Turbines to Brazil’s Bioenergy

General Electric Co. (GE) , the U.S. power-equipment supplier, will sell 1.8 billion reais ($895.3 million) of turbines to Bioenergy Geradora de Energia Ltda., a Brazilian company seeking to develop the world’s cheapest wind energy. GE will deliver 207 turbines to 13 of Bioenergy’s projects in the northeastern state of Maranhao and will furnish another 170 others for future projects, Sao Paulo-based Bioenergy said today in an e-mailed statement. GE will invest in a domestic supply chain to be eligible for a new local-content requirement of national development lender Banco Nacional de Desenvolvimento Economico e Social so that its turbines can be sold with cheap debt, according to the statement. Bioenergy agreed to sell energy from wind farms in a government-organized auction in December at 87.77 reais a megawatt-hour, the world’s lowest price for wind power, Eduardo Tabbush, an analyst at Bloomberg New Energy Finance, said Jan. 31 in a telephone interview.


Magnum Rises After Eagle Ford Asset Sale: Dallas Mover

Magnum Hunter Resources Corp. (MHR) , the Houston-based oil and natural gas producer with operations in West Virginia , North Dakota and Saskatchewan, advanced the most in more than four months after it agreed to sell Texas assets for double the amount invested in them. Magnum Hunter rose 5.6 percent to $4.02 at 11:10 a.m. in New York after earlier gaining 12 percent, the most intraday since Nov. 19. The shares have fallen 38 percent in the last year. Magnum Hunter sold mineral leases on 19,000 net acres in the Eagle Ford Shale for $401 million in cash, including production equivalent to about 3,000 barrels of oil a day to Penn Virgina Corp., according to statements today by the companies. The deal is expected to close in mid May. Penn Virginia , based in Radnor, Pennsylvania, fell 0.3 percent today. The company also announced it will sell $400 million of senior notes due 2020 to finance the purchase, which will increase its Eagle Ford holdings to about 54,000 net acres. The sale price was double the capital invested in the leases during the past three years, Magnum Hunter Chairman and Chief Executive Officer Gary C. Evans said in the company’s statement today. The company is keeping 7,000 net acres in the Eagle Ford deposit.


Bega Jumps as Fonterra Buys Stake in Cheese Bidder: Sydney Mover

Bega Cheese Ltd. (BGA) , the under bidder in a three-way contest for an Australian peer, rose to a record in Sydney trading after New Zealand ’s Fonterra Cooperative Group acquired a 6 percent stake. Bega jumped as much as 12 percent, before trading 8.6 percent higher at A$4.79 at 10:17 a.m. local time. Fonterra, the world’s largest dairy exporter, said it paid A$4.95 a share for a 6 percent stake in New South Wales state-based Bega after the market closed yesterday. The purchase raised the value of Bega’s cash and share offer for Warrnambool Cheese & Butter Factory Co. (WCB) to a peak of A$7.93, just 7 cents less than the current highest bid from Montreal-based Saputo Inc. (SAP) Murray Goulburn Cooperative Co. is also bidding for Warrnambool. “There has recently been a lot of consolidation activity in the Australian dairy industry ,” Fonterra Chief Executive Officer Theo Spierings said today in a statement. “It is important that Fonterra participates, and we have confidence in Bega and the strategy it is pursuing.” Bega’s board is meeting next week to discuss raising its offer for Warrnambool after its proposal was cleared yesterday by the Australian Competition and Consumer Commission, Chairman Barry Irvin said in an interview yesterday. Bega had climbed 40 percent at the close of trading yesterday since it made a takeover bid on Sept. 12 for the maker of brands including Sungold milk.


Chrysler Reports Production in April Increased 13%

Chrysler Group LLC, moving past costly rollouts of new Ram pickups and Jeep sport-utility vehicles, said production rebounded in April for the No. 3 U.S. automaker’s first monthly increase since November. Chrysler’s North American output rose 13 percent to 216,758 cars, SUVs and trucks, from 192,491 a year earlier, the Auburn Hills , Michigan-based company said on its website. Production in April was the highest since Fiat SpA-controlled Chrysler built 220,101 vehicles in August. Sergio Marchionne , chief executive officer of Chrysler and Fiat, is relying on Chrysler and its 37 straight monthly U.S. sales gains, to offset losses at Turin, Italy-based Fiat’s mass- market brands in Europe. Rising production should help Chrysler rebound from a 65 percent decline in first-quarter profit, which was dragged down by introductions of the new Ram Heavy Duty truck and Jeep Grand Cherokee and Compass SUVs. Automakers record revenue when vehicles are shipped to dealers. “The best way to look at this is a one-time event, it’s a one-off; just close your eyes, plug your nose and move it on from here,” Marchionne told analysts on an April 29 conference call. Chrysler earlier this week said its April U.S. sales rose 11 percent, topping the 10 percent increase that was the average of 11 analysts’ estimates in a Bloomberg survey. Chrysler’s share of the U.S. market has risen for three straight years, to 11.4 percent in 2012, according to researcher Autodata Corp. First-quarter net income slid to $166 million from $473 million a year earlier. Chrysler held to its forecast for annual revenue of $72 billion to $75 billion and net income of about $2.2 billion.


Google Poised to End U.S. Antitrust Probe With Agreement

The U.S. is set to close its antitrust probe of Google Inc. (GOOG) with an agreement letter from the company that addresses the use of other websites’ content and the ability of advertisers to transfer data to other platforms, a person familiar with the matter said. The Federal Trade Commission announced it will hold a press conference today about the Google investigation at 1 p.m. The Google letter, which the agency will describe as enforceable by law, will offer other websites the option to say they don’t want their content to be displayed in Google search results, said the person, who asked not to be named because details of the FTC’s decision aren’t public. The letter will also include a non-discrimination provision on Google’s advertising practices, allowing advertisers to compare data from their campaigns across platforms, the person said. The decision to close the probe of Google’s search practices without enforcement action is a blow to competitors including Microsoft Corp. (MSFT) , Yelp Inc. (YELP) , and Expedia Inc. (EXPE) An alliance of such e-commerce and Web-search companies pressed the agency to bring a lawsuit, claiming Google’s dominance of Internet search, combined with favoring its own services in answers to queries, violates antitrust laws. Unprecedented Step Matt Reilly, a former FTC litigator who represents Fairsearch.org, the alliance of Google competitors, said it was unprecedented for the agency to resolve a case with a voluntary agreement. “Any time we were negotiating and the parties would say ‘Can we do this without a consent decree?’ the answer was always ‘No,’” Reilly said. “Now, everybody is going to want the deal Google got. There is a lot of disappointment.” Google is expected to settle allegations it misused patents to thwart competitors in smartphone technology, three people familiar with the matter have said.


GSW Accepts Deutsche Wohnen’s $2.3 Billion Bid for Landlord

GSW Immobilien AG (GIB) recommended that shareholders accept a takeover bid by Deutsche Wohnen AG (DWNI) that values GSW at about 1.7 billion euros ($2.3 billion). The deal would create Germany ’s second-largest owner of apartments. Deutsche Wohnen Chief Executive Officer Michael Zahn will head the new company, while GSW co-CEOs Joerg Schwagenscheidt and Andreas Segal will become chief operating officer and chief financial officer, respectively, GSW said in a statement after the market closed yesterday. “There are opportunities that we can bring out with a combination,” Segal said by phone today. “We won’t be competing against each other any more.” GSW shareholders must decide whether to accept an offer of 2.55 Deutsche Wohnen shares for each GSW share by midnight on Oct. 30. Both companies are based in Berlin, where price and rent increases have outpaced the rest of the country. Deutsche Wohnen needs acceptances from investors owning at least 75 percent of GSW shares to proceed with the deal. The combined company would own about 150,000 apartments valued at 8.5 billion euros, Deutsche Wohnen said in August. Deutsche Annington Immobilien SE (ANN) is Germany’s biggest residential landlord, with 180,000 apartments and a market value of 4.4 billion euros. Dividend Policy Starting in 2014, Deutsche Wohnen will increase its dividend to about 60 percent of funds from operations excluding sales, a measure of a property company’s ability to generate cash, according to yesterday’s statement. The new company would be able to become more profitable because of cost savings and an increase in negotiating power when acquiring apartments or borrowing money, Segal said. “Being bigger creates advantages,” he said. “We are optimistic that we can optimize our financing options further.” The merger will lead to cost savings of about 25 million euros per year starting in about two years, when the integration is complete, Deutsche Wohnen said on Aug. 20. Cost savings may also come from job cuts, Schwagenscheidt said by phone. “We don’t rule out that personnel may be reduced,” he said.


NewStar Financial Said to Work With Credit Suisse on Sale

NewStar Financial Inc. (NEWS) , the business lender backed by private-equity firms Corsair Capital LLC and Capital Z Partners Management LLC, is exploring a sale, according to people with knowledge of the matter. NewStar is working with Credit Suisse Group AG (CSGN) to solicit offers, said one of the people, who asked not to be identified because the process is confidential. Boston-based NewStar rose 13 percent to $17.76 today, its highest since February 2007, giving the company a market value of about $863 million. Specialty lenders like NewStar, which makes loans to health-care, manufacturing and energy companies, typically charge higher interest rates, making them appealing targets for U.S. banks looking to increase profits. CapitalSource Inc. (CSE) , which also lends to businesses, agreed in July to sell to PacWest Bancorp (PACW) for $2.3 billion. “This company is a prime takeover target,” said Sameer Gokhale, an analyst who covers NewStar for Janney Montgomery Scott LLC in Philadelphia. NewStar will appeal to regional banks interested in expanding in commercial lending, said Gokhale, who has a “neutral” rating on the stock and a price target of $17. Logical buyers include Jacksonville, Florida-based EverBank Financial Corp. (EVER) , Miami Lakes, Florida-based BankUnited Inc. (BKU) , and Wayzata, Minnesota-based TCF Financial Corp. (TCB) , he said. Early Backers Robert Brown , a spokesman for NewStar, declined to comment, as did Credit Suisse spokesman Jack Grone and Robert Siegfried at Corsair. Spokesmen at Capital Z and the three regional banks didn’t immediately reply to requests for comment. NewStar, with offices in nine U.S. cities, makes loans and leases of $5 million to $20 million to companies with annual sales of $25 million to $500 million, according to its annual report. The company had assets of $2.28 billion at the end of June, data compiled by Bloomberg show. Along with other backers, Corsair and Capital Z, which each own close to 20 percent of NewStar, invested $210 million in equity and $450 million in debt to help start the company in 2004, according to a press release at the time. PacWest offered about 1.47 times CapitalSource’s book value, in the cash-and-stock takeover that was announced in July. At that multiple, NewStar would be worth about $18.21 a share, based on its book value of $12.39 a share at the end of June, data compiled by Bloomberg show.


Drillsearch to Study More Acquisitions After Acer Energy Deal

Drillsearch Energy Ltd. (DLS) , BG Group Plc (BG/) ’s shale gas exploration partner in central Australia, will consider further acquisitions in the region after its purchase last year of Acer Energy Ltd. Drillsearch, which acquired the rest of Acer in a deal that valued the Brisbane-based company at A$132 million ($126 million), is studying more purchases in the Cooper Basin to complement its holdings in the area, Managing Director Brad Lingo said yesterday in an interview at a conference in Brisbane. “It would be clearly focused on adding reserves, adding production and adding cash flow , and focused around quality development,” he said, declining to identify any assets. “There are from time to time those opportunities.” Drillsearch is among explorers in Australia’s Cooper Basin seeking to benefit from Asia’s demand for liquefied natural gas, which has driven more than $60 billion of export projects being built on the east coast. The Sydney-based company has 15 permits in the outback region straddling the border of South Australia and Queensland states, operating a dozen. A need for funding to expand may drive further consolidation and partnerships in the industry, Lingo said. “There are companies who have, and there are the have nots,” he said. “There are companies out there that have good high-quality underlying assets, but don’t have the reserves, production or cash flow. It’s our understanding they are finding it quite challenging to attract capital.”


Harvest Natural Surges on Gabon Offshore Oil Discovery

Harvest Natural Resources Inc. (HNR) , the oil explorer with operations in Venezuela, Indonesia and Oman, climbed the most in more than six months after reporting its second crude discovery off the coast of Gabon. Harvest advanced 11 percent to $10.25 at the close in New York , the most since June 22, and its highest since Nov. 8, 2011. Harvest found oil in the Dussafu Tortue Marin-1 well, the Houston-based company said in a statement today. The well, in 380 feet (116 meters) of water, was dug to a depth of more than 2 miles. Further appraisal is needed to determine the scope of the discovery, including fluid sampling and a sidetrack well, according to the statement. Harvest operates Dussafu and holds a 66.66 percent stake. Panoro Energy ASA (PEN) , owner of 33.33 percent of Dussafu, rose 20 percent in Oslo to close at 3.35 kroner, the biggest one-day gain since it was spun off by Norse Energy Corp ASA (NEC) in June, 2010. Harvest reported its first discovery off Gabon in June 2011.


Companies in U.S. Added More Workers Than Forecast in June

Companies in the U.S. added more workers than economists projected in June as businesses prepared for a pickup in consumer demand in the second half of the year. The 188,000 increase in employment followed a revised 134,000 gain in May that was 1,000 less than initially estimated, the ADP Research Institute reported today. Based in Roseland, New Jersey, the institute is a division of Automatic Data Processing Inc., a company that manages employer payrolls. The median forecast of 38 economists surveyed by Bloomberg called for an advance of 160,000 in June. Companies added workers last month as home prices strengthened and Americans continued to buy cars , electronics and other goods in the face of higher taxes that took effect in January. A Labor Department report on July 5 may show private payrolls rose by 175,000 last month, according to the Bloomberg survey median forecast. “The job market continues to gracefully navigate through the strongly blowing fiscal headwinds,” Mark Zandi, chief economist at Moody’s Analytics Inc. in West Chester, Pennsylvania , said in a statement. Moody’s produces the figures with ADP. Today’s report shows that employment advances are “broad-based across industries and businesses of all sizes,” he said. Estimates in the Bloomberg survey ranged from gains of 100,000 to 196,000. May’s figure was revised from a previously reported increase of 135,000. Factory Work Manufacturers, construction companies and other goods-producing industries increased payrolls by 27,000. Construction employment rose by 21,000 and factories gained 1,000 jobs, today’s report showed. Payrolls at service providers climbed by 161,000. Companies employing more than 499 workers added 49,000 jobs. Employment at businesses with 50 to 499 employees rose 55,000 and the smallest companies increased payrolls by 84,000, the report showed. The Labor Department may report in two days that private and government payrolls climbed 165,000 in June after a 175,000 increase in May, according to the Bloomberg survey median. The unemployment rate fell to 7.5 percent, economists projected. ADP in October changed the method it uses to calculate its employment figures dating back to 2001. The report now is derived from a larger sample and is released jointly with Moody’s Analytics.


Samsung Boosting Venture-Capital Funding to $1.1 Billion

Samsung Electronics Co. (005930) , the world’s largest maker of mobile phones, said it was boosting venture- capital funding by $100 million to help create new businesses for its unit making memory chips and electronics parts. The company unveiled the Catalyst Fund for early stage startups, as well as a strategy and innovation center in Menlo Park , California , at an event near the site on Sand Hill Road. The company and affiliates also run the $1 billion Samsung Ventures America Fund. “We must reach out to global hot spots and global talent,” Young Sohn, president and chief strategy officer of device solutions for Samsung, said at the event. “This is the epicenter of disruptive forces.” Like other companies such as Intel Corp. (INTC) , Google Inc. and Microsoft Corp., Samsung is trying to use its cash to encourage the development of technology that will benefit its business in the future. Traditional venture-capital investors aren’t investing in basic science and technology startups, creating a gap that Samsung intends to help fill, Sohn said. Samsung’s venture arm, now headquartered in Seoul, has more than $1 billion in assets under management and did 20 deals worth $160 million last year, Sohn said. Investments by the new funds will be in the areas of Internet-connected devices, mobile health, computer-human interface, cloud-data centers and mobile privacy, he said. Shares of the Suwon, South Korea-based company fell 0.1 percent to 1,435,000 won in South Korean trading today.


Chrysler Names Bigland to Head Ram After Diaz’s Defection

Chrysler Group LLC, the U.S. automaker majority owned by Fiat SpA (F) , moved Reid Bigland from chief executive officer of the Dodge brand to the Ram truck business, replacing Fred Diaz, who left to join Nissan Motor (7201) Co. Bigland, who started with Chrysler in 2006, retains his roles as head of U.S. sales and CEO of Chrysler Canada. Tim Kuniskis, a 21-year Chrysler employee and former leader of the Fiat brand in North America, will succeed Bigland as Dodge brand chief, the Auburn Hills , Michigan-based company said in a statement. The moves come as Chrysler tries to maintain its streak of 36 consecutive monthly U.S. sales gains, the longest stretch in company records that date back to 1985, Chrysler surprised analysts last year when it was the only major U.S. automaker to gain share in its home market. Jason Stoicevich, a 10-year Chrysler veteran, will take over as head of Fiat for North America and continue to run the company’s California office. Bruno Cattori was named head of Chrysler’s operations in Mexico, another post held by Diaz, who spent 24 years at Chrysler. Diaz begins May 1 as vice president of sales and marketing for the Nissan brand in the U.S., the Yokohama, Japan-based company said last week.


Notore, ContourGlobal Plan $600 Million Nigerian Power Plant

Notore Chemical Industries Ltd. , a Nigeria fertilizer producer, is investing $600 million to build a power plant at its factory in the southern town of Onne, Ikponwosa Izedonmwen, the company’s head of corporate finance, said. The 500-megawatt plant will be built jointly with ContourGlobal LP, a New York-based investment company specializing in the electric-power industry, Izedonmwen said on May 6 in an e-mailed response to questions. Financing will come from export credit agencies and development banks, with local lenders making up the balance, he said. “Repayment of the loan/investment will come solely from the cash flow of the project being funded, not from other assets of the project sponsors,” Izedonmwen said. He declined to provide the names of lenders involved. ContourGlobal didn’t immediately respond to e-mails and a call seeking comment. Blackouts are a daily occurrence in Nigeria, Africa ’s most populous country with more than 160 million people. Electricity demand is almost double the supply of about 4,000 megawatts. Nigeria plans to boost electricity output to 10,000 megawatts by next year and double that by 2016, Power Minister Chinedu Nebo said on April 22. South Africa , with less than a third of Nigeria’s population, has the capacity to generate about 40,000 megawatts of power. The government is offering incentives to investors, including five-year tax breaks and permission to charge higher electricity tariffs, according to the Abuja-based Bureau of Public Enterprises. Notore has an agreement with Japan ’s Mitsubishi Corp. (8058) to build a new plant at its Onne complex, according to its website. The company currently operates a fertilizer plant with capacity to produce 350,000 metric tons of ammonia a year, 500,000 tons of urea and 650,000 tons of a “bulk blend” of nitrogen, phosphorus and potassium, or NPK fertilizer. Construction of the power plant will start in the second-half of next year and it is expected to be completed by the third quarter of 2016.


U.S. Video-Game Sales Fell 25 Percent in February

U.S. retail sales of video-game software, hardware and accessories declined 25 percent last month to $810 million, according to researcher NPD Group Inc. Hardware sales fell the most, shrinking 36 percent to $244.2 million from the previous year, the Port Washington , New York-based company said today in an e-mailed statement. Software sales for the current generation of home consoles and portable devices fell 27 percent to $369.9 million. Sales of physical game products are declining amid a consumer shift toward mobile play on smartphones and tablets. Retail sales now account for about 50 percent of total U.S. consumer spending on games, according to Liam Callahan, an NPD analyst. Software sales fell even with the release of two shooter games from Electronic Arts Inc. (EA) and one from Sega in February, NPD reported. Callahan estimated total US. video-game sales were almost $1.2 billion in February. Nintendo Co. (7974) in November released its first home console since 2006, and Sony Corp. (6758) said last month its new PlayStation 4 will go on sale in time for year-end holiday shoppers. Both give consumers the ability to download games over the Internet. Microsoft Corp. earlier today said U.S. retailers sold 302,000 Xbox 360 games consoles in February, citing NPD data. Its machines account for 41 percent of current-generation console sales, it said in a statement. Microsoft sold 426,000 Xbox 360 units a year ago. Sony and Nintendo do not typically release the console sales data.


Zynga Rises on Speculation Yahoo Might Consider Buyout

Zynga Inc., the biggest maker of online social games, climbed to the highest price since July on speculation that Yahoo! Inc. might consider buying the company. The shares rose 10 percent to $3.91 at the close in New York. The stock has advanced 67 percent this year, compared with a 9.4 percent gain for the Russell 1000 Index. Yahoo may consider buying Zynga as Chief Executive Officer Marissa Mayer seeks acquisitions to bolster the U.S. Web portal’s mobile and social capabilities, Blake Harper, an analyst at Wunderlich Securities Inc., wrote in a research report today. Since Mayer arrived, Yahoo has ceded share in its core business of display advertising amid mounting competition from Google Inc. and Facebook Inc. With a market value of about $3 billion, Zynga might be an attractive target if Mayer opts for deals valued at more than $1 billion, Harper said. Yahoo might also look at purchasing food- review sites Yelp Inc. (YELP) and OpenTable Inc. (OPEN) , or Millennial Media Inc. (MM) , which offers mobile advertising, Harper said. Representatives from Sunnyvale, California-based Yahoo and Zynga, based in San Francisco, declined to comment. Yelp, OpenTable and Millennial Media also declined to comment.


Iberdrola Starts Three U.S. Wind Plants Totaling 265 MW

Iberdrola SA (IBE) , Spain’s biggest utility, started up three U.S. wind farms with a combined electricity generation capacity of more than 265 megawatts. The Manzana park in California is among the biggest operated by the Bilbao, Spain-based company, with 126 General Electric Co. (GE) 1.5-megawatt turbines providing capacity of 189 megawatts, Iberdrola said today in an e-mailed statement. The other two farms are the 48-megawatt Groton site in New Hampshire with 24 Gamesa Corp. Tecnologica SA machines, and the 28.5-megawatt Hoosac plant in Massachusetts with 19 GE turbines. Iberdrola, the second-biggest operator of wind farms in the U.S. with more than 5,700 megawatts of installed capacity, says it’s a “strategic” market and it will use 1.7 billion euros ($2.3 billion) or 16 percent of its investment from 2012 through 2014 in the U.S., mainly on grid and renewable energy projects.


Lego Beats Hasbro as World’s No. 2 Toymaker Amid Asia Sales Jump

Lego A/S, the Danish building-block maker, said first-half sales jumped 13 percent with the strongest growth in Asia amid a slower start to 2013 in the toy markets of North America and Europe. Lego’s first-half revenue rose to 10.4 billion kroner ($1.84 billion) from 9.13 billion kroner in the same period a year earlier, according to a statement published today by the Billund, Denmark-based company. Lego, which estimates the world’s children spend 5 billion hours a year playing with its toys, said it won market share on rising sales of its new building-block line Legends of Chima and its Lego Friends sets. The latter series, introduced in 2012, is Lego’s sixth attempt over the years to target girls and the “most significant” new product in a decade, Chief Executive Officer Joergen Vig Knudstorp has said. “It’s a very satisfactory result in view of general developments in the world toy market which has had a slow start to the year in the largest and most mature markets of North America , Europe and Japan ,” Knudstorp said in today’s statement. Sales growth in Lego Friends was “significantly higher” than the company average, Lego said. Legends of Chima, introduced this year, has become one of Lego’s largest products and the company said it expects sales will continue to grow. The range features animal tribes battling for control of a world and offers a number of different sets. Higher Share Lego, which is closely held, said it boosted its global market share to 8.8 percent in the first half from 8.6 percent at the end of last year. The company is controlled by Denmark’s richest man Kjeld Kirk Kristiansen, the grandson of Lego founder Ole Kirk Kristiansen, who has a net worth of $6.4 billion, according to Bloomberg rankings. Lego’s first-half sales were higher than those of Hasbro Inc. (HAS) The Pawtucket, Rhode Island-based toymaker reported sales of $1.43 billion for the same six-month period on July 22. Mattel Inc. (MAT) , the world’s largest toymaker which is based in El Segundo, California , on July 17 reported a 4 percent advance in first half revenue to $2.16 billion. Lego’s net income grew 18 percent in the first six month of 2013 to 2.38 billion kroner, making the company bigger than both Mattel and Hasbro in terms of profit.


Einhorn’s Greenlight Critizized by Investor on Apple

David Einhorn ’s Greenlight Capital Inc. was criticized by an investor who said he’s dismayed by the hedge fund’s opposition to an Apple Inc. (AAPL) shareholder resolution. Greenlight is trying to block a proposal that would keep Apple from issuing preferred shares without shareholder approval, saying the resolution would tie the company’s hands. The effort undermines shareholder rights, the Nathan Cummings Foundation, which invests in Greenlight Capital through a pooled investment vehicle, wrote in a letter to Greenlight President Einhorn. The letter was obtained by Bloomberg. “By insisting that the company retain the ability to issue preferred shares -- for whatever purpose -- without a shareholder vote, you undermine shareholders’ rights,” Nathan Cummings Foundation Chief Executive Officer Simon Greer wrote in the letter. Greenlight’s public criticism of the resolution reinvigorated debate over how Apple should use its $137.1 billion in cash and securities. Apple executives, who reinstated dividends last year and are using stock buybacks, have said they’re considering ways to return more of those holdings. Einhorn is urging Apple to issue preferred shares. Bill Dempsey, Nathan Cummings Foundation’s senior vice president for finance, confirmed the letter, and said that his foundation invests in Greenlight through a so-called fund of funds. ‘Former Investor’ “This is a former investor who redeemed in 2010,” Greenlight Capital said in an e-mailed statement. “We wish them well.” Einhorn is out of touch with who his own investors are, Greer said in an e-mailed statement. “We’d be happy to meet with Mr. Einhorn and invite along the other investors in his fund who he seems to dismiss,” Greer said in the statement. “In the meantime, we hope Mr. Einhorn will consider withdrawing his losing bid to deny Apple shareholders their rights.” Steve Dowling, a spokesman for Apple, declined to comment. He referred to a statement issued by the company earlier this month. “We will thoroughly evaluate Greenlight Capital’s current proposal to issue some form of preferred stock,” Apple said Feb. 7. “We welcome Greenlight’s views and the views of all of our shareholders.” The Nathan Cummings Foundation’s letter to Einhorn was reported earlier by the Financial Times. Apple declined less than 1 percent to $446.89 at 9:37 a.m. in New York.


Airbus Throws Out Workshare Rule as Government Involvement Wanes

Airbus SAS said it won’t return to a previous practice of apportioning work on new programs by national ownership structure, adding to signs that the European manufacturer is emancipating itself from political meddling. Airbus first dropped the practice on its A350 wide-body jet, where the fact that France got more work so riled the German government that it continues to withhold some financial aid. Even so, the Toulouse, France-based company will continue in its new vein, said Tom Enders, the chief executive officer of Airbus parent European Aeronautic, Defence & Space Co. “We broke with previous decades’ practice of work-share being assigned according to an artificial kind of national split,” Ender told journalists in Paris. “It was the first program where we looked at what is the best, most competitive supplier, and that’s certainly something we’ll insist on going forward.” The comment highlights a liberation from government ties that Enders, a German and vocal critic of political involvement in his company has sought to push through. Airbus contributes 80 percent of revenue to the parent company, and Enders sought to balance out civil and defense sales last year with a merger with BAE Systems Plc (BA/) , which failed because of opposition from Germany ’s government. German Complaint Germany has complained that Airbus’s factories there were given less work than those in France on the long-range A350 jet, due to enter service in 2014. Germany has pledged 1.1 billion euros ($1.56 billion) toward the A350, or one third of the contribution made together with France and the U.K. The A350’s development costs amount to about 11 billion euros. EADS, also based in Toulouse, has overhauled its shareholder structure since the collapse of the BAE discussions late last year. Germany, which previously held no direct stake in the company, gained a 12 percent to match France’s reduced holding. At the same time, governments no longer have the special veto rights that they possessed under the old structure.


Boeing Hikes 20-Year Pilot-Hiring Forecast 8.3% as Airlines Grow

Boeing Co. (BA) , the world’s largest aerospace manufacturer, predicts global airlines will need to hire 498,000 new pilots to keep pace with aircraft orders over the next 20 years, 8.3 percent more than it forecast a year ago. The hiring will be greatest among Asian carriers, which will need to recruit 192,300 pilots through 2032, Chicago-based Boeing said in a study released today. That’s 3.6 percent more than the planemaker predicted in its 2012 forecast. Pilot training among the region’s rapidly growing carriers has been in the spotlight as U.S. crash investigators examine the manual flying skills and cockpit teamwork of pilots guiding an Asiana Airlines Inc. jet that crashed in San Francisco in July. “The urgent demand for competent aviation personnel is a global issue that is here now and is very real,” Sherry Carbary, vice president of Boeing Flight Services, said in a statement. The report comes as Boeing opens its training center for pilots of its 787 Dreamliner today at Miami’s international airport. The transfer of the training center from the Seattle area will result in the loss of about 34 union jobs, said Bill Dugovich, spokesman for the 26,000-member Professional Aerospace Union, known as SPEEA. Boeing predicts airlines will hire 556,000 maintenance technicians over the next two decades, 7.5 percent fewer than a year ago, as carriers retire aging aircraft in favor of new models that require fewer overhauls.


Renault to Increase Turkey Production on Recovering Local Demand

Renault SA (RNO) , whose plant in Turkey makes more than half the cars manufactured in the country, plans to increase local production to take advantage of a market expected to grow slightly compared with last year. Oyak-Renault Otomobil Fabrikalari AS , a joint venture between Renault and Turkey’s military pension fund Oyak Group, will try to use as much as 92 percent of the 360,000 vehicles-a- year capacity at the Bursa plant this year, compared with almost 86 percent in 2012, Tarik Tunalioglu, chief executive of the joint venture, said in a news conference in Istanbul today. “We want to increase production because we are expecting a bigger demand for our new models this year, especially Clio 4,” Tunalioglu said. “More than half of our production this year will be the Clio 4 model.” Turkish car and van sales are expected to grow about 3 percent to 780,000 this year, said Ibrahim Aybar, chief executive of Renault Mais Motorlu Araclar Imal & Satis AS, the sales joint venture between Renault and Oyak Group. Turkey ’s total vehicle market contracted 11.5 percent last year after a September tax increase on cars combined with slower economic growth to curb consumer demand. Market Share Renault’s 2012 sales in Turkey were $4.2 billion, Tunalioglu said, without giving a year-on-year comparison. Exports fell slightly to $3.15 billion from $3.21 billion in 2011. Renault produces sedan models of Fluence, Clio 4 and Megane at the Turkish plant. Renault Mais, which also sells imported Dacia models, expects to introduce new models in Turkey in order to increase sales from 118,000 units last year, Aybar said. “We want to increase our market share in Turkey this year from 13.1 percent in 2012,” he said. Renault will start selling nine new models in the Turkish market this year including Scenic, Latitude, Dacia Sandero and electric vehicle Zoe, Aybar said. Oyak-Renault will continue its three-shift production at the plant, Tunalioglu said, while competitor Tofas Turk Otomobil Fabrikasi AS (TOASO) , a joint venture made up of Fiat SpA (F) and Koc Holding AS (KCHOL) , said Jan. 2 it will cut shifts to two because of weak demand from European markets. Toyota Motor Corp. (7203) , Honda Motor Co. (7267) and Hyundai Motor Co. (005380) are among manufacturers of cars, vans, trucks and buses that sell local and imported brands in Turkey. The country’s total motor vehicle sales fell to 815,000 units in 2012, said Mustafa Bayraktar, head of Turkey’s Car Distributors’ Association. Renault will invest 55 million euros ($73 million) in the Bursa plant, compared with $212 million last year, Tunalioglu said.


Aemetis to Use Chromatin’s Sorghum at California Ethanol Plant

Aemetis Inc. , a U.S. ethanol producer, agreed to convert sorghum crops from closely held Chromatin Inc. into biofuel at a plant in California. Farmers are expected to plant the hybrid grain sorghum on as many as 30,000 acres near Aemetis’ plant in the town of Keyes, Chicago-based Chromatin said today in a statement. Shifting some production from corn to sorghum will enable a portion of Aemetis’s ethanol to qualify as an advanced biofuel, which has at least half the lifecycle greenhouse-gas emissions of gasoline, according to the U.S. Environmental Protection Agency. U.S. gasoline and diesel producers must blend 2.75 billion gallons (10.5 billion liters) of advanced biofuels with their products in 2013 as part of a broader 16.6 billion-gallon requirement that also includes conventional corn-based ethanol and biodiesel. Aemetis announced plans to use sorghum at the idled 60 million-gallon a year plant last month. The agreement is Chromatin’s second with a U.S. ethanol producer this week. It said Feb. 18 that Pacific Ethanol Inc. agreed to use a similar amount of its crops to produce ethanol. Aemetis rose about 15.4 percent to 60 cents at 2:27 p.m. in New York.


Nike’s Second in Command to Retire Amid Management Shift

Nike Inc. (NKE) , the world’s largest sporting-goods maker, announced a slew of management changes that included repositioning several current executives and the retirement of its second-in-command. Charlie Denson , a 34-year veteran and president of the Nike brand, will retire in January and be replaced from within the company by Trevor Edwards on July 1, Beaverton, Oregon-based Nike said yesterday in a statement. Nike also promoted Eric Sprunk to the new position of chief operating officer. Edwards has “some very big shoes to fill, but they have a very strong, deep management team,” said Brian Yarbrough, an analyst with Edward Jones & Co. in St. Louis who has a hold rating on the company’s shares. While Nike’s sales rebounded after the recession, with revenue surging 16 percent to $24.1 billion in the fiscal year ended in May 2012, growth has slowed in the past three quarters. Revenue in China , Nike’s second-largest market, has declined and the company replaced the head of its business there in May. Nike fell 0.3 percent to $60.57 at the close in New York. The shares have gained 17 percent this year, compared with a 12 percent advance for the Standard & Poor’s 500 Index. Denson started at the company in 1979 as an assistant manager of one of its original stores. From there, he worked his way up in the sales department and eventually became head of the Nike brand and co-president of the company with Mark Parker in 2001. Parker then became chief executive officer in 2006, replacing William Perez. Sprunk, currently executive vice president of merchandising and product, will lead all manufacturing, sourcing and information technology for the company. He joined the company in 1993. Retail Stores Edwards, currently executive vice president of brand and category management, will succeed Denson as Nike brand president with expanded responsibilities that include overseeing the company’s retail stores and e-commerce. Edwards has been with Nike since 1992. Those two areas were previously run by Jeanne Jackson in her role as president of direct-to-consumer. Jackson moves into another new position, president of product and merchandising. The former Wal-Mart Stores Inc. and Gap Inc. executive will oversee the creation of all footwear, apparel and equipment. She joined Nike management in 2009 after eight years on the board. Nike also moved Thomas Clarke to president of innovation, a new position, from president of new business development. Gary DeStefano, president of global operations, will retire on July 29 after 31 years at the company. His responsibilities will be taken by Elliott Hill as president of geographies and sales and by Sprunk and Edwards.


We can now add the tool's output to the message chain with `ToolMessage`, so the model can answer based on the retrieved documents.

*Note: Here we use the base model instead of the RAG model to limit our agent to one retrieval call. A fully autonomous agent could decide to make subsequent calls to best answer the request*

In [213]:
from langchain_core.messages import ToolMessage

In [214]:
# Message list with the retrieved documents for the base model
messages = [
    # RAG System prompt
    SystemMessage(RAG_PROMPT),
    # Client request
    HumanMessage(request),
    # RAG model response
    rag_response,
    # Retrieval tool response
    ToolMessage(documents_str, tool_call_id=tool_call["id"]),
]

# Invoke the base model with the messages
response = base_model.invoke(messages)

BadRequestError: Error code: 400 - {'error': {'message': "An assistant message with 'tool_calls' must be followed by tool messages responding to each 'tool_call_id'. The following tool_call_ids did not have response messages: call_u3u6Hl1CNl3RLL3U1RfILeP3, call_Kfu3RJET995xEQmpYxizqgoZ, call_dnQXVbmA271Fsy6S3GJBjS0f, call_n3tup0QumrOoun8UtpI0CuAX, call_PwsOXsyJHkMBlhF7IPCwLtQN, call_0xT5XEjRATftNZf5mHD11ZfA", 'type': 'invalid_request_error', 'param': 'messages', 'code': None}}

In [ ]:
Markdown(response.content)

## Practical Tips

- Watch the temperature setting: Lower values (like 0) are usually better for factual responses
- Pay attention to the number of retrieved documents (`RETRIEVAL_K`): More isn't always better
- The system prompt is crucial: It sets the context and behavior of your bot

## Conclusion

You just learned how to create a chatbot and augment it with a retrieval tool using **LangChain**, this concludes the first part of our workshop!

In the next section, we'll discover **LangGraph** and show how it allows to build sophisticated and flexible LLMs workflows.